In [7]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import matplotlib.dates as mdates
from matplotlib.patches import Rectangle
import time

def fetch_data():
    try:
        end_date = datetime.now()
        start_date = end_date - timedelta(days=365)
        start_timestamp = int(start_date.timestamp())
        end_timestamp = int(end_date.timestamp())
        url = "https://query1.finance.yahoo.com/v8/finance/chart/BTC-EUR"
        params = {
            'period1': start_timestamp,
            'period2': end_timestamp,
            'interval': '1d',
            'events': 'history'
        }
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        print("fatching BTC/EUR data from Yahoo Finance")
        response = requests.get(url, params=params, headers=headers)
        if response.status_code != 200:
            print(f"request fail: {response.status_code}")
            return pd.DataFrame()
        data = response.json()
        if 'chart' not in data or 'result' not in data['chart'] or not data['chart']['result']:
            print("APIwrong")
            return pd.DataFrame()
        chart_data = data['chart']['result'][0]
        timestamps = chart_data.get('timestamp', [])
        quotes = chart_data.get('indicators', {}).get('quote', [{}])[0]
        if not timestamps or not quotes:
            print("API wrong")
            return pd.DataFrame()
        df = pd.DataFrame({
            'Date': [datetime.utcfromtimestamp(ts) for ts in timestamps],
            'Open': quotes.get('open', [np.nan] * len(timestamps)),
            'High': quotes.get('high', [np.nan] * len(timestamps)),
            'Low': quotes.get('low', [np.nan] * len(timestamps)),
            'Close': quotes.get('close', [np.nan] * len(timestamps)),
            'Volume': quotes.get('volume', [np.nan] * len(timestamps))
        })
        df = df.dropna() 
        if df.empty:
            print("empty")
            return pd.DataFrame()
        df.set_index('Date', inplace=True)
        print(f"Successfully obtained {len(df)} data ({df.index[0].strftime('%Y-%m-%d')} 到 {df.index[-1].strftime('%Y-%m-%d')})")
        return df
    
    except Exception as e:
        print(f"wrong: {str(e)}")
        return pd.DataFrame()

def save_to_csv(data, filename='BTC_EUR_1Y_Historical.csv'):
    if not data.empty:
        data.to_csv(filename)
        print(f"datasaveto: {filename}")
    else:
        print("no data to save")

def plot_candlestick(data):
    if data.empty:
        print("no data to draw")
        return
    fig, ax = plt.subplots(figsize=(14, 8))
    plt.title('BTC/EUR for one year', fontsize=16, pad=20)
    ax.set_ylabel('prize (€)', fontsize=12)
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    plt.xticks(rotation=45)
    ax.grid(True, linestyle='--', alpha=0.6)
    width = 0.8 / np.sqrt(len(data))
    dates_num = mdates.date2num(data.index)

    for i in range(len(data)):
        date_num = dates_num[i]
        open_price = data['Open'][i]
        close_price = data['Close'][i]
        high_price = data['High'][i]
        low_price = data['Low'][i]
        color = 'green' if close_price >= open_price else 'red'
        ax.plot([date_num, date_num], [low_price, high_price], 
                color='black', linewidth=0.8)
        body_bottom = min(open_price, close_price)
        body_height = abs(close_price - open_price)
        rect = Rectangle(
            (date_num - width/2, body_bottom),
            width, 
            body_height,
            facecolor=color,
            edgecolor='black',
            linewidth=0.8
        )
        ax.add_patch(rect)
    ax_volume = ax.twinx()
    ax_volume.set_ylabel('Turnover', fontsize=12)
    volume_colors = ['green' if data['Close'][i] >= data['Open'][i] else 'red' 
                    for i in range(len(data))]
    ax_volume.bar(dates_num, data['Volume'], 
                 color=volume_colors,
                 width=width * 1.5, 
                 alpha=0.4)
    ax_volume.grid(False)
    price_min = data['Low'].min() * 0.98
    price_max = data['High'].max() * 1.02
    ax.set_ylim(price_min, price_max)
    ax.set_xlim(dates_num[0] - 1, dates_num[-1] + 1)
    plt.tight_layout()
    plt.savefig('BTC_EUR_Candlestick_Chart.png', dpi=300, bbox_inches='tight')
    print("save as: BTC_EUR_Candlestick_Chart.png")
    plt.show()
if __name__ == "__main__":
    start_time = time.time()
    print("Start obtaining BTC/EUR historical data")
    btc_data = fetch_data()
    if not btc_data.empty:
        print("\n数据摘要:")
        print(f"Date Range: {btc_data.index[0]} to {btc_data.index[-1]}")
        print(f"Average closing price: {btc_data['Close'].mean():.2f} €")
        print(f"maximum price: {btc_data['High'].max():.2f} €")
        print(f"bottom price: {btc_data['Low'].min():.2f} €")
        save_to_csv(btc_data)

        plot_candlestick(btc_data)

开始获取BTC/EUR历史数据...
正在从Yahoo Finance获取BTC/EUR数据...
请求失败，状态码: 403
未能获取数据，请检查网络连接或重试
